# Initiation au CNN avec Pytorch

Ce TP s'effectue individuellement ou en binome. Veuillez respecter les consignes suivantes pour le rendu de votre travail :

* Renommez le selon le format suivant : "DL_2023_TP_Torch_prenom1_nom1_prenom2_nom2.ipynb".
* Veillez à ce que votre nom et prénom soient complétés dans la cellule ci-dessous.
* Veillez à avoir bien exécuté toutes les cellules de code et que les résultats soient tous bien visible dans le notebook sans nécessiter une ré-exécution.
* Partagez le notebook avec hana.sebia@univ-lyon1.fr

Veuillez compléter vos noms et prénoms ci-dessous :

*   **Prenom 1** : Kévin
*   **Nom 1** : TANG
*   **Prenom 2** : Yann
*   **Nom 2** : VINCENT

Ce TP est une introduction au framework Pytorch. Nous allons construire une des premières architectures de CNN présenté par [Yann Le Cun](https://fr.wikipedia.org/wiki/Yann_Le_Cun), un [LeNet](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf).

L'architecture du LeNet est détaillée dans la figure ci-dessous:

![leNet5.jpeg](leNet5.jpeg "Architecture Lenet")

In [1]:
# on importe les bibliothèques pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print ("CUDA device not found.")

CUDA device not found.


## Chargement du jeu de données
La tâche que nous souhaitons réaliser est la classification d'image [MNIST](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST). La base de données MNIST (Modified National Institute of Standards and Technology) est une base de données de chiffres écrits à la main. C'est un jeu de données très utlisé en apprentissage automatique. Il regroupe 60000 images d'apprentissage et 10000 images de test. On peut télécharger ces données à partir du module dataset de torchvision en séparant le chargement du train/test set. Il est également possible d'appliquer un ensemble de transformations aux images dès le chargement.

In [3]:
from torchvision import datasets, transforms # On peut importer directement le dataset de pytorch

# On définit transforms qui permet de redimensionner l'image en 32*32 et de la transformer en tensor
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])

# On télécharge et on créer la dataset d'entraienement à l'aide du module datasets de torchvision
train_dataset = datasets.MNIST(root='mnist_data',
                               train=True,
                               transform=transforms,
                               download=True)

# On télécharge et on créer la dataset de test à l'aide du module datasets de torchvision
valid_dataset = datasets.MNIST(root='mnist_data',
                               train=False,
                               transform=transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 65426989.36it/s]


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 26489327.32it/s]


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 121827286.44it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 11082332.03it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



Une fois les train/test sets chargés, on définit des dataloaders qui permettent de créer des batchs pour la phase train de l'apprentissage de notre modèle comme suit :

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32 #taille du batch size

# On définit le data loaders d'entraienement . Le data loaders permet de créer des batchs. On doit lui renseigner le batch size.
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
# On définit le data loaders de validation .
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False)

## Définition du modèle

---
<span style='color:Green'>**Question**</span>

Implémenter la classe LeNet avec l'architecture proposée en utilisant l’interface [nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) de PyTorch:

---


Dans l'initialisation de la classe LeNet
1. La première couche [convolutive](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) avec 6 noyaux de taille 5×5 et le stride de 1.
2. Une couche de [sous-échantillonnage/mise](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html) en commun avec 6 noyaux de taille 2×2.
3. La deuxième couche convolutive avec la même configuration que la première, cette fois avec 16 filtres. La sortie de cette couche est de 10×10×16.
4. La deuxième couche de mise en commun. La logique est identique à celle de la précédente, mais cette fois, la couche comporte 16 filtres. La sortie de cette couche est de taille 5×5×16.
5. La dernière couche convolutive avec 120 noyaux 5×5.
6. La dernière couche est un réseau de [neurones simple](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) comme détaillé dans l'architecture ci-dessus.

Toute couche convolutive doit être suivi d'une [ normalisation](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html#torch.nn.BatchNorm2d) et d'une fonction d'activation [ReLu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#relu)

Dans la fonction forward de la classe LeNet définissez le passage de la donnée x en appliquant à la fin un [softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) pour calculer la probabilité d'appartenance à la classe des chiffres de mnist.


**Indice**
Indice: l’utilisation de la méthode [.view()](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html) ou de la couche [nn.Flatten()](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) peut être utile pour ré-arranger les tenseurs avant ou après les couches linéaires. Par exemple, x.view(-1, 1, 28, 28) permet de transformer un tenseur de dimensions 784 en un tenseur de dimensions (batch, 1, 28, 28)…

In [5]:

class LeNet(nn.Module): # On créer la classe LeNet qui hérite de la classe mère Module

    def __init__(self): # On définit
        super(LeNet, self).__init__()

        self.feature_extractor = nn.Sequential(
            # Couche 1
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),

            # Couche 2
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Couche 3
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            # Couche 4
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Couche 5
            nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(120),
            nn.ReLU(),
        )

         ## La dernière couche est un réseau de neurones simple
        self.classifier = nn.Sequential(
           #
           #
           #
            nn.Linear(120 * 1 * 1, 84),  # Ajustez la taille d'entrée en fonction de la sortie de la dernière couche convolutive
            nn.ReLU(),
            nn.Linear(84, 10),  # 10 pour la classification des chiffres de 0 à 9 dans MNIST
        )

    def forward(self, x): # on défini le passage de nos données

        x = self.feature_extractor(x)            # On applique l'extracteur
        x = x.view(-1, 120 * 1 * 1)            # On réarrange la structure de x
        logits = self.classifier(x)       # Prédire la classe des images dans x
        probs = nn.functional.softmax(logits, dim=1)        # calculer les probabilités d'appartenance à partir des prédictions
        return logits, probs


net = LeNet()
print(net) # On peut afficher les paramètres du modèle

LeNet(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (9): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)


---
<span style='color:Green'>**Question**</span>

Compléter et commenter la fonction train(chaque ligne du code), celle-ci permet d'entrainer votre modèle:

---

In [6]:
# On créer la fonction qui permet d'entrainer le modèle

def train(train_loader, model, criterion, optimizer):
    '''
    Function for the training step of the training loop
    '''
    model.train() # entrainement du model
    running_loss = 0 # initialisation de la loss

    for X, y_true in train_loader:

        optimizer.zero_grad() # réinitilisation des gradients

        X = X.to(device) # on envoie les données X sur la GPU
        y_true = y_true.to(device) # on envoie les données Y sur la GPU

        # Forward pass (on passe les données dans le modèle)
        y_hat, _ = model(X)
        loss = criterion(y_hat, y_true) # Calcule la perte entre les prédictions et les vraies étiquettes

        # Rétropropagation du gradient
        loss.backward()
        optimizer.step()  # Applique une étape d'optimisation pour mettre à jour les poids du modèle

        running_loss += loss.item()

    running_loss /= len(train_loader) # Calcule la loss moyenne pour cet epoch
    return model, optimizer, running_loss # Retourne le modèle mis à jour, l'optimiseur et la perte moyenne

---
<span style='color:Green'>**Question**</span>

En vous inspirant de la fonction train, completer la fonction validate, celle-ci permet de tester votre modèle:

---

In [7]:
# On créer la fonction qui permet de valider le modèle

def validate(valid_loader, model, criterion):
    '''
    Function for the validation step of the training loop
    '''

    model.eval()
    running_loss = 0

    for X, y_true in valid_loader:

        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X)
        loss = criterion(y_hat, y_true)
        running_loss += loss.item()

    running_loss /= len(valid_loader)
    return model, running_loss

---
<span style='color:Green'>**Question**</span>

Ecrivez la fonction training_loop qui prend en paramètre le model, le criterion, l'optimizer, le train_loder, le valid_loader et le nombre d'épochs. Cette fonction permet de faire une étape de train et une étape de validate par epoch. Affichez l'erreur d'apprentissage et de validation toutes les 5 épochs.

---
---
**Note**
Lors de la validation, les gradients ne doivent pas être modifiés ([.no_grad](https://pytorch.org/docs/stable/generated/torch.no_grad.html)).

---

In [8]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, print_every=5):
    model.to(device)

    for epoch in range(epochs):
        # Entraînement
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer)

        # Validation
        model, valid_loss = validate(valid_loader, model, criterion)

        # Affichage des résultats toutes les 'print_every' époques
        if (epoch + 1) % print_every == 0 or epoch == 0:
            print(f'Epoch {epoch + 1}/{epochs} => '
                  f'Training Loss: {train_loss:.4f}, '
                  f'Validation Loss: {valid_loss:.4f}')

    return model, optimizer

---
<span style='color:Green'>**Question**</span>

Définissez un [optimizer]( https://pytorch.org/docs/stable/optim.html#module-torch.optim) ainsi qu'une fonction de perte adaptés et justifiez votre choix.

---

In [13]:
import torch.optim as optim

model = net.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss() # Pour les problemes de classification : CrossEntropy


# L'optimiseur Adam a été choisi en raison de sa capacité à gérer efficacement
# les jeux de données volumineux et les modèles complexes, grâce à ses taux d'apprentissage adaptatifs.

# La fonction de perte CrossEntropyLoss est appropriée pour les problèmes de classification,
# car elle mesure l'écart entre la distribution de probabilité prédite par le modèle et la distribution réelle des étiquettes,
# favorisant ainsi une convergence efficace.

---
<span style='color:Green'>**Question**</span>

Lancez l'entrainement de votre modèle en choisissant un nombre d'epoch judicieusement.

---

In [12]:
N_EPOCHS = 25
model, optimizer = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, print_every=5)

Epoch 1/25 => Training Loss: 0.0301, Validation Loss: 0.0334
Epoch 5/25 => Training Loss: 0.0192, Validation Loss: 0.0305
Epoch 10/25 => Training Loss: 0.0135, Validation Loss: 0.0280
Epoch 15/25 => Training Loss: 0.0092, Validation Loss: 0.0333
Epoch 20/25 => Training Loss: 0.0078, Validation Loss: 0.0285
Epoch 25/25 => Training Loss: 0.0066, Validation Loss: 0.0397
